In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [9]:
superStoreDF = spark.read.csv("file:///C:/data/SuperStore/SuperStore_US_Sales.csv", header = True, inferSchema = True)

In [10]:
superStoreDF.printSchema()
superStoreDF.show(2)

root
 |-- Order_ID: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub_Category: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantitty: integer (nullable = true)
 |-- Region: string (nullable = true)

+--------------+----------+-------+---------------+------------+------+---------+-------+
|      Order_ID| OrderDate|   City|       Category|Sub_Category| Sales|Quantitty| Region|
+--------------+----------+-------+---------------+------------+------+---------+-------+
|CA-2017-107727|10/19/2017|Houston|Office Supplies|       Paper|29.472|        3|Central|
|US-2017-118038| 12/9/2017|Houston|Office Supplies|     Binders| 1.248|        3|Central|
+--------------+----------+-------+---------------+------------+------+---------+-------+
only showing top 2 rows



In [3]:
managerInfoDF = spark.read.csv("file:///C:/data/SuperStore/managerByRegion.csv", header = True, inferSchema = True)

In [4]:
managerInfoDF.show(2)

+-------------+------+
|       Person|Region|
+-------------+------+
|Anna Andreadi|  West|
|  Chuck Magee|  East|
+-------------+------+
only showing top 2 rows



In [5]:
returnInfoDF = spark.read.csv("file:///C:/data/SuperStore/totalReturns.csv", header = True, inferSchema = True)
returnInfoDF.show(2)

+--------+--------------+
|Returned|      Order ID|
+--------+--------------+
|     Yes|CA-2017-153822|
|     Yes|CA-2017-129707|
+--------+--------------+
only showing top 2 rows



In [11]:
superStoreDF.createOrReplaceGlobalTempView("SuperStore123")
superStoreDF.select("*").show()

+--------------+----------+-------------+---------------+------------+-------+---------+-------+
|      Order_ID| OrderDate|         City|       Category|Sub_Category|  Sales|Quantitty| Region|
+--------------+----------+-------------+---------------+------------+-------+---------+-------+
|CA-2017-107727|10/19/2017|      Houston|Office Supplies|       Paper| 29.472|        3|Central|
|US-2017-118038| 12/9/2017|      Houston|Office Supplies|     Binders|  1.248|        3|Central|
|US-2017-118038| 12/9/2017|      Houston|      Furniture| Furnishings|  9.708|        3|Central|
|US-2017-118038| 12/9/2017|      Houston|Office Supplies|     Storage|  27.24|        3|Central|
|US-2017-119662|11/13/2017|      Chicago|Office Supplies|     Storage|230.376|        3|Central|
|CA-2017-155558|10/26/2017|    Rochester|     Technology| Accessories|  19.99|        1|Central|
|CA-2017-155558|10/26/2017|    Rochester|Office Supplies|      Labels|   6.16|        2|Central|
|US-2017-109484| 11/6/2017|   

In [21]:
#OrDate,City, sum(Sales) from superStore group by OrderDate,City").show()

In [17]:
superStoreDF.registerTempTable("ss1")
sqlContext.sql("select orderDate, city, sales, rank() over (partition by orderDate order by sales desc) rnk1 from (select orderDate, city, sum(sales) as sales from ss1 group by orderDate, city order by orderDate) as tmp ").where("rnk1 = 1").show(10)

+----------+-------------+--------+----+
| orderDate|         city|   sales|rnk1|
+----------+-------------+--------+----+
| 10/1/2017|New York City| 1704.89|   1|
|10/10/2017|       Aurora| 239.358|   1|
|10/12/2017|     Lakewood| 2258.28|   1|
|10/13/2017|   Sacramento|5325.884|   1|
|10/14/2017|San Francisco|   46.36|   1|
|10/15/2017|  Los Angeles|  715.19|   1|
|10/16/2017|     Columbia|1893.786|   1|
|10/17/2017|      Warwick|   62.92|   1|
|10/19/2017|   Burlington|1633.188|   1|
| 10/2/2017|    Knoxville|2437.672|   1|
+----------+-------------+--------+----+
only showing top 10 rows



In [18]:
superStoreBASEDF = sqlContext.sql("select orderDate, city, sum(sales) as sales from ss1 group by orderDate, city order by orderDate")

In [19]:
superStoreBASEDF.printSchema()

root
 |-- orderDate: string (nullable = true)
 |-- city: string (nullable = true)
 |-- sales: double (nullable = true)



In [21]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window.partitionBy("orderDate").orderBy(desc("sales")).rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [34]:
from pyspark.sql.functions import max
from pyspark.sql.functions import dense_rank, rank
from pyspark.sql.functions import expr, col, column, desc, expr


In [24]:
salesRank = rank().over(windowSpec)

In [37]:
superStoreBASEDF.orderBy(desc("orderDate")).select("orderDate", "city", expr("round(sales,2)").alias("total_sales"), salesRank.alias("sales_ranker")).show()

+---------+-------------+-----------+------------+
|orderDate|         city|total_sales|sales_ranker|
+---------+-------------+-----------+------------+
|12/9/2017|     Richmond|    1827.51|           1|
|12/9/2017|      Detroit|    1115.68|           2|
|12/9/2017|  Los Angeles|    1096.49|           3|
|12/9/2017|     Whittier|     444.77|           4|
|12/9/2017|      Phoenix|     432.41|           5|
|12/9/2017|      Salinas|     148.26|           6|
|12/9/2017|   Park Ridge|     143.86|           7|
|12/9/2017|   Cincinnati|      94.78|           8|
|12/9/2017| Grand Rapids|       54.9|           9|
|12/9/2017| Indianapolis|      43.85|          10|
|12/9/2017|      Houston|       38.2|          11|
|12/9/2017|      Chicago|      15.98|          12|
|12/9/2017| Philadelphia|      13.71|          13|
|12/8/2017|San Francisco|    4268.57|           1|
|12/8/2017|      Madison|    1816.36|           2|
|12/8/2017|     Cranston|     592.74|           3|
|12/8/2017|New York City|     4

In [38]:
superStoreBASEDF.orderBy(desc("orderDate")).select("orderDate", "city", expr("round(sales,2)").alias("total_sales"), salesRank.alias("sales_ranker")).explain()

== Physical Plan ==
*Project [orderDate#162, city#163, total_sales#525, sales_ranker#526]
+- Window [rank(sales#388) windowspecdefinition(orderDate#162, sales#388 DESC NULLS LAST, ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS sales_ranker#526], [orderDate#162], [sales#388 DESC NULLS LAST]
   +- *Sort [orderDate#162 ASC NULLS FIRST, sales#388 DESC NULLS LAST], false, 0
      +- *Project [orderDate#162, city#163, round(sales#388, 2) AS total_sales#525, sales#388]
         +- *Sort [orderDate#162 DESC NULLS LAST], true, 0
            +- Exchange rangepartitioning(orderDate#162 DESC NULLS LAST, 200)
               +- *Sort [orderDate#162 ASC NULLS FIRST], true, 0
                  +- Exchange rangepartitioning(orderDate#162 ASC NULLS FIRST, 200)
                     +- *HashAggregate(keys=[orderDate#162, city#163], functions=[sum(sales#166)])
                        +- Exchange hashpartitioning(orderDate#162, city#163, 200)
                           +- *HashAggregate(keys=[orderDat

In [40]:
superStoreBASEDF.orderBy(desc("orderDate")).select("orderDate", "city", expr("round(sales,2)").alias("total_sales"), salesRank.alias("sales_ranker")).where("sales_ranker = 1").show(5)

+---------+-------------+-----------+------------+
|orderDate|         city|total_sales|sales_ranker|
+---------+-------------+-----------+------------+
|12/9/2017|     Richmond|    1827.51|           1|
|12/8/2017|San Francisco|    4268.57|           1|
|12/7/2017|      Chicago|    1889.99|           1|
|12/6/2017|    Brentwood|      10.68|           1|
|12/5/2017|      Seattle|     497.04|           1|
+---------+-------------+-----------+------------+
only showing top 5 rows



In [41]:
superStoreDF.printSchema()

root
 |-- Order_ID: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub_Category: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantitty: integer (nullable = true)
 |-- Region: string (nullable = true)



In [59]:
superStoreDF.groupby("orderDate","city").sum("Sales").select("orderDate", "city", col("sum(Sales)").alias("sales")).orderBy("orderDate").show()

""" 
select orderDate, city, sum(sales) as sales from ss1 group by orderDate, city order by orderDate
"""


+----------+-------------+------------------+
| orderDate|         city|             sales|
+----------+-------------+------------------+
| 10/1/2017|San Francisco|            104.75|
| 10/1/2017|      Fremont|            104.85|
| 10/1/2017|     San Jose|           413.784|
| 10/1/2017|  San Antonio|             6.672|
| 10/1/2017|New York City|           1704.89|
| 10/1/2017|      Seattle|            643.52|
|10/10/2017|       Aurora|           239.358|
|10/12/2017|       Monroe|             7.968|
|10/12/2017|New York City|              9.96|
|10/12/2017|     Rockford|           254.604|
|10/12/2017|     Wheeling|           673.344|
|10/12/2017|  Los Angeles|441.71999999999997|
|10/12/2017|     Lakewood|           2258.28|
|10/12/2017|San Francisco|            102.83|
|10/12/2017|       Dallas|1211.7759999999998|
|10/12/2017|  Albuquerque|           674.872|
|10/13/2017|   Louisville|            348.56|
|10/13/2017| Philadelphia|           114.376|
|10/13/2017|    Lafayette|        

' \nselect orderDate, city, sum(sales) as sales from ss1 group by orderDate, city order by orderDate\n'